<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/semantic_vector4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Pretrained sentence-transformers models](https://www.sbert.net/docs/pretrained_models.html)

[Our chosen light model in hugging face](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

[ Convert a collection of raw documents to a matrix of TF-IDF features ](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
)

In [ ]:
!pip install -U sentence-transformers
!pip install stop_words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util  #util for importing cosine similarity
import numpy as np
import re #for tokenizing logs
import pandas as pd
import stop_words